In [34]:
import timm
import torch
import openpyxl
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import torchvision.transforms as transforms
import cv2
import os ,sys
import numpy as np
# checkpoint = torch.load('D:/dataset_Huo_1/output/resnet18_1-1704.pth',map_location='cpu')
# checkpoint = checkpoint['state_dict']
model = timm.create_model(model_name='resnet18',num_classes = 2,checkpoint_path=path_to_patch-level_models)
# params=model.state_dict()
# for k,v in checkpoint.items():
#     print(k) #只打印key值，不打印具体参数。
# model.load_state_dict(checkpoint,strict=True)
model = model.eval()

In [35]:
def searchFile(key,startPath = '.'):
    if not os.path.isdir(startPath):
        raise ValueError
    l= [os.path.join(startPath,x) for x in os.listdir(startPath)]  #列出所有文件的绝对路径
    #listdir出来的相对路径 不能用于 isfile  abspath只能用在当前目录
    filelist=[x for x in l if os.path.isfile(x) if key in os.path.splitext(os.path.basename(x))[0]] #文件
    #只查找文件名中  不包括后缀 文件路径
    if not hasattr(searchFile,'basePath'):#把函数当成类 添加属性
        searchFile.basePath=startPath #只有第一次调用才会赋值给basePath
    outmap = map(lambda x:os.path.relpath(x,searchFile.basePath),filelist) #转换成相对于初始路径的相对路径

    outlist = list(outmap) 

    dirlist= [x for x in l if os.path.isdir(x)]  #目录
    for dir in dirlist:
        outlist = outlist + searchFile(key,dir)
 
    return outlist

In [ ]:
#the files named path_to_names_of_patients are placed in the folder /MAg/name_patient/
info = openpyxl.load_workbook(path_to_names_of_patients)
inf = info.active
# number = openpyxl.load_workbook('D:/dataset_Huo_1/MSIMUT_train.xlsx')
# num = number.active
from openpyxl import Workbook
wb = Workbook()
sheet = wb.active
sheet.title = "MSI_patient_data"

feature_MSI = Workbook()
sheet2 = feature_MSI.active
sheet2.title = "feature_MSI"

patch_info = Workbook()
sheet3 = patch_info.active
sheet.title = "patch_info"

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
right_num = 0
vote=[]
# Please adjust the parameters of the for loop according to the number of patients in each set
for i in range(0,12):
    patient_name = inf.cell(row=1, column=(i+1)).value
    print(i,"patient id:",patient_name)
    sheet["A%d" % (i+1)].value = patient_name
    # image_num = inf.cell(i+1,1).value
    file_names = searchFile(patient_name,startPath = path_images)
    image_num = len(file_names)
    print("numbers of images of this patient",image_num)
    pred_MSS = 0
    pred_MSI = 0
    patient_patch = []
    if image_num:
        for j in range(image_num):
            path = 'D:/dataset_Huo_4/STAD/validation/MSIMUT/' + file_names[j] #filename of images
            image = Image.open(path).convert('RGB')
            tensor = transform(image).unsqueeze(0)
            with torch.no_grad():
                out = model(tensor)
            probabilities = torch.nn.functional.softmax(out[0], dim=0)
            probabilities = np.array(probabilities)
            if probabilities[0] >= probabilities[1]:
                pred_MSI = pred_MSI + 1
            elif probabilities[0] < probabilities[1]:
                pred_MSS = pred_MSS + 1
            sheet3.cell(j+1,i+1).value = probabilities[0]
            patient_patch.append(probabilities[0])
            # print(j,'current patch:',probabilities[0])
        counts,bins = np.histogram(patient_patch,bins = 10,range = (0,1))
        counts_norm = counts / image_num
        for t in range(10):
            sheet2.cell(i+1,t+1).value = counts_norm[t]
        vote_buf = pred_MSI / (pred_MSI + pred_MSS)
        sheet2.cell(i+1,11).value = vote_buf
        sheet2.cell(i+1,12).value = 1
        if vote_buf >= 0.5:
            right_num = right_num + 1
        vote.append(vote_buf)
        print(i, "pred_MSI:",pred_MSI,"pred_MSS:",pred_MSS,"vote_MSI:",vote[i],"mean vote:",sum(vote)/(i+1))
        print("\n")
        sheet["B%d" % (i+1)].value = vote[i]
        sheet["C%d" % (i+1)].value = pred_MSI
        sheet["D%d" % (i+1)].value = pred_MSS
print('total number: 12','right number:',right_num)
# wb.save('D:/dataset_Huo_4/resnet18_0925/MSI_classfication.xlsx')
feature_MSI.save(path_to_hist_features)
patch_info.save(path_to_results_of_patches)